In [1]:
using Distributed

In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
using CSV, OnlineStats, Base.Threads, Random, Distributions, Plots
@everywhere using LinearAlgebra, SparseArrays, SharedArrays

filepath = "/Users/guillaume/Downloads/ml-latest-small/ratings.csv"

In [17]:
filepath = "/Users/guillaume/Downloads/ml-latest/ratings.csv"

"/Users/guillaume/Downloads/ml-latest/ratings.csv"

In [18]:
f = CSV.File(filepath, use_mmap=true)

CSV.File("/Users/guillaume/Downloads/ml-latest/ratings.csv", rows=27753444):
Tables.Schema:
 :userId     Union{Missing, Int64}  
 :movieId    Union{Missing, Int64}  
 :rating     Union{Missing, Float64}
 :timestamp  Union{Missing, Int64}  

In [7]:
mutable struct Rating
    itemId::Int64
    userId::Int64
    value::Float64
end

In [19]:
# TODO: Pre-allocate A, b and result
# TODO: use --track-allocation=user
@everywhere function lsq(F::DenseArray{Float64,2}, rows::Vector{Int64}, ratings::Vector{Float64}, μ::Float64, reg::Float64)::Vector{Float64}
    @views A::Matrix{Float64} = [ones(length(rows)) F[rows, 2:end]]
    @views b::Vector{Float64} = ratings .- μ .- F[rows, 1]
    Symmetric(A'A + reg*I) \ (A'b)
end

@inline function predict(i::Int64, u::Int64, μ::Float64, P::DenseArray{Float64,2}, Q::DenseArray{Float64,2})::Float64
    @views μ + P[i,1] + Q[u,1] + dot(P[i,2:end],Q[u,2:end])
end

function cost(R::AbstractArray{Float64,2}, P::DenseArray{Float64,2}, Q::DenseArray{Float64,2}, μ::Float64=mean(nonzeros(R)))::Float64
    map(zip(findnz(R)...)) do (i,u,r)
        abs2(predict(i,u,μ,P,Q) - r)
    end |> sum
end

function als_threads(ratings, k::Int64=10;
        nepochs::Int64=10,
        reg::Float64=0.0,
        cb::Union{Nothing, Function}=nothing)
    
    o = Mean()
    R::SparseMatrixCSC{Float64,Int64} = let items::Vector{Int64} = Int64[],
                                            users::Vector{Int64} = Int64[],
                                            values::Vector{Float64} = Float64[]
        for r::Rating in ratings
            push!(items, r.itemId)
            push!(users, r.userId)
            push!(values, r.value)
            fit!(o, r.value)
        end
        sparse(items, users, values)
    end
    μ::Float64 = value(o)
    
    P::Matrix{Float64} = [zeros(R.m) rand(Normal(0.0, 1e-4), R.m, k)]
    Q::Matrix{Float64} = [zeros(R.n) rand(Normal(0.0, 1e-4), R.n, k)]
    
    rated_items::Vector{Int64} = unique(sort(findnz(R)[1]))
    rating_users::Vector{Int64} = unique(sort(findnz(R)[2]))
    
    for epoch::Int64 in 1:nepochs
        @threads for u::Int64 in rating_users
            items_rated_by_user::Vector{Int64}, ratings_given_by_user::Vector{Float64} = findnz(R[:,u])
            q::Vector{Float64} = lsq(P, items_rated_by_user, ratings_given_by_user, μ, reg)
            @views Q[u,:] .= q[:]
        end
        
        @threads for i::Int64 in rated_items
            users_who_rated_item::Vector{Int64}, ratings_given_to_item::Vector{Float64} = findnz(R[i,:])
            p::Vector{Float64} = lsq(Q, users_who_rated_item, ratings_given_to_item, μ, reg)
            @views P[i,:] .= p[:]
        end

        if cb !== nothing
            cb(epoch, cost(R,P,Q,μ))
        end
    end
    
    return P, Q
end

@everywhere @inline function lsq(P::SharedMatrix{Float64}, Q::SharedMatrix{Float64}, T::Matrix{Float64},
        u::Int64, nz::Tuple{Vector{Int64},Vector{Float64}}, μ::Float64, reg::Float64)::Vector{Float64}
    @views begin
        A = T[1:length(nz[1]), 2:end]
        A[:, 2:end] = P[nz[1], 2:end]
        b = T[1:length(nz[1]), 1]
        b[:] = nz[2] .- μ .- P[nz[1], 1]
        Q[u,:] .= cholesky(A'A + reg*I) \ (A'b)
    end
end

function als_processes(ratings, k::Int64=10;
        nepochs::Int64=10,
        reg::Float64=0.0,
        cb::Union{Nothing, Function}=nothing)
    
    println("Parsing ratings...")
    o = Mean()
    R::SparseMatrixCSC{Float64,Int64} = let items::Vector{Int64} = Int64[],
                                            users::Vector{Int64} = Int64[],
                                            values::Vector{Float64} = Float64[]
        for r::Rating in ratings
            push!(items, r.itemId)
            push!(users, r.userId)
            push!(values, r.value)
            fit!(o, r.value)
        end
        sparse(items, users, values)
    end
    μ::Float64 = value(o)
    
    println("Creating matrices...")
    P::SharedMatrix{Float64} = SharedMatrix{Float64}([zeros(R.m) rand(Normal(0.0, 1e-4), R.m, k)])
    Q::SharedMatrix{Float64} = SharedMatrix{Float64}([zeros(R.n) rand(Normal(0.0, 1e-4), R.n, k)])
    
    rated_items::Vector{Int64} = unique(sort(findnz(R)[1]))
    rating_users::Vector{Int64} = unique(sort(findnz(R)[2]))
    
    @everywhere workers() T = ones(max($(R.m),$(R.n)), $(k)+2)
    
    println("Compute...")
    for epoch::Int64 in 1:nepochs
        @sync @distributed for u::Int64 in rating_users
            lsq(P, Q, T, u, findnz(R[:,u]), μ, reg)
        end
        
        @sync @distributed for i::Int64 in rated_items
            lsq(Q, P, T, i, findnz(R[i,:]), μ, reg)
        end

        if cb !== nothing
            cb(epoch, cost(R,P,Q,μ))
        end
    end
    
    return P, Q
end

als_processes (generic function with 2 methods)

In [ ]:
let
    costs = []
    ratings = (Rating(r.movieId, r.userId, r.rating) for r in f)

    @time P, Q = als_threads(ratings, 100;
        nepochs=10,
        reg=0.001,
        cb=(epoch, cost)->begin
            IJulia.clear_output(true)
            println("epoch: $(epoch), cost: $(cost)")
            push!(costs, cost)
            end)

    plot(costs)
end

In [ ]:
let
    costs = []
    ratings = (Rating(r.movieId, r.userId, r.rating) for r in f)
    
    @time P, Q = als_processes(ratings, 100;
        nepochs=10,
        reg=0.001,
        cb=(epoch, cost)->begin
            IJulia.clear_output(true)
            println("epoch: $(epoch), cost: $(cost)")
            push!(costs, cost)
            end)

    plot(costs)
end

Parsing ratings...
Creating matrices...
Compute...


In [ ]:
rmprocs(workers)

costs = []
ratings = (Rating(r.movieId, r.userId, r.rating) for r in f)
    
@time P, Q = alsbiased(ratings, 100;
    nepochs=10,
    reg=0.001)